In [1]:
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv() 

True

## Setting up Llama index 

In [2]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# set the default embedding model to BAAI/bge-small-en-v1.5
Settings.embed_model = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")

In [4]:
# setting Ollama 
import os
from llama_index.llms.ollama import Ollama
# ollama_url = os.getenv("OLLAMA_URL") # to be set to the Ollama URL
# Settings.llm = Ollama(model="llama2", base_url=ollama_url, request_timeout=10)
Settings.llm = Ollama(model="llama2")


In [5]:

from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama2")

In [6]:
resp = llm.complete("Who is Paul Graham?")
print(resp)

ReadTimeout: timed out

In [5]:
# RAG pipeline
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("data/examples/paul_graham_essays").load_data()
index = VectorStoreIndex.from_documents(documents)

In [8]:
index.__dict__

{'_use_async': False,
 '_store_nodes_override': False,
 '_embed_model': HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x00000130D5CFFF40>, max_length=512, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None),
 '_insert_batch_size': 2048,
 '_storage_context': StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x00000130D8FF8040>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x00000130D8FF83A0>, vector_stores={'default': <llama_index.core.vector_stores.simple.SimpleVectorStore object at 0x00000130D9671AB0>, 'image': <llama_index.core.vector_stores.simple.SimpleVectorStore object at 0x00000130D96719F0>}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x00000130D8FF8CA0>),
 '_service_context': None,
 '_docstore': <llama_i

In [22]:
from llama_index.core import Response
from tonic_validate import CallbackLLMResponse

query_engine = index.as_query_engine()

# Gets the response from llama index in a format Tonic Validate can understand
def get_llama_response(prompt) -> CallbackLLMResponse:
    response = query_engine.query(prompt)
    # Check response is of type Response
    if not isinstance(response, Response):
        raise ValueError(f"Expected Response, got {type(response)}")
    
    # Get the response and context from the Llama index
    context = [x.text for x in response.source_nodes]
    answer = response.response
    if answer is None:
        raise ValueError("No response from Llama")
    
    return {
        "llm_answer": answer,
        "llm_context_list": context
    }

## Asking Questions to Llama Index

In [15]:
import json
qa_pairs = []
with open("./data/examples/question_and_answer_list.json", "r") as qa_file:
    qa_pairs = json.load(qa_file)

In [16]:
qa_pairs

[{'question': 'What makes Sam Altman a good founder?',
  'answer': 'He has a great force of will.',
  'reference_article': 'Five Founders',
  'reference_text': '5. Sam Altman\n\nI was told I shouldn\'t mention founders of YC-funded companies in this list. But Sam Altman can\'t be stopped by such flimsy rules. If he wants to be on this list, he\'s going to be.\n\nHonestly, Sam is, along with Steve Jobs, the founder I refer to most when I\'m advising startups. On questions of design, I ask "What would Steve do?" but on questions of strategy or ambition I ask "What would Sama do?"\n\nWhat I learned from meeting Sama is that the doctrine of the elect applies to startups. It applies way less than most people think: startup investing does not consist of trying to pick winners the way you might in a horse race. But there are a few people with such force of will that they\'re going to get whatever they want.'},
 {'question': 'When was the essay "Five Founders" written?',
  'answer': 'April 200

In [19]:
def print_qa_pair(qa_pair):
    print(f"Question: {qa_pair['question']}")
    print(f"Answer: {qa_pair['answer']}")
    print()

for qa_pair in qa_pairs:
    print_qa_pair(qa_pair)

Question: What makes Sam Altman a good founder?
Answer: He has a great force of will.

Question: When was the essay "Five Founders" written?
Answer: April 2009

Question: When does the most dramatic growth happen for a startup?
Answer: When the startup only has three or four people.

Question: What is the problem with business culture versus start up culture with respect to productivity?
Answer: In business culture, energy is expended on outward appearance to the detriment of productivity, while in startup culture there is no value of appearance it's all about productivity.

Question: What's the single biggest thing the government could do to increase the number of startups in this country?
Answer: Establish a new class of visa for startup founders.

Question: How could one create a rigorous government definition of what a startup is to categorize whether a business is a startup?
Answer: One could define a startup as a company that has received investment by recognized startup investor

In [20]:
example_qa = qa_pairs[0]
print_qa_pair(example_qa)

Question: What makes Sam Altman a good founder?
Answer: He has a great force of will.



In [23]:
get_llama_response(example_qa["question"])

ReadTimeout: timed out

## Vectara & Tonic Validate

In [7]:
from llama_index.indices.managed.vectara import VectaraIndex
from llama_index.core import SimpleDirectoryReader

# Tonic Validate

In [ ]:
api_key_header = {
    "customer-id": 43437896,
    "x-api-key": "zwt_ApbPSAKP2pwNkh7GwzOg7dNP2f94ng3G9_3FLA"
}